In [2]:
import pandas as pd
import numpy as np
import time
import re
import os
import json
import requests
from nameparser import HumanName

In [3]:
keys_json = json.load(open('env_keys.json'))
scopus_key = keys_json['scopus_key']
aditya_key = keys_json['aditya_imperial_scopus_key']
req_headers = {
    'X-ELS-APIKey' : scopus_key,
    # 'X-ELS-APIKey' : aditya_key
}


In [4]:
print('READING IN AUTHOR-ABSTRACT DATA')
authors_abstracts_df = pd.DataFrame()
filenames = os.listdir('scopus_data')
path_prepend = 'scopus_data/'
for filename in filenames:
    if re.search(r'_author_abstract_funding\.csv', filename):
        file_path = path_prepend + filename
        journal_authors_abstracts_df = pd.read_csv(file_path)
        print(file_path, len(journal_authors_abstracts_df))
        authors_abstracts_df = pd.concat([authors_abstracts_df, journal_authors_abstracts_df])

print('Author-Abstract observations: {}'.format(len(authors_abstracts_df)))
unique_authors_df = authors_abstracts_df.drop_duplicates(subset=['sc_author_id', 'sc_author_given_name', 'sc_author_last_name', 'sc_author_indexed_name'])
print('Unique Authors (from author-abstract data): {}'.format(len(unique_authors_df)))

READING IN AUTHOR-ABSTRACT DATA
scopus_data/AER_author_abstract_funding.csv 10904
scopus_data/ALJ_author_abstract_funding.csv 646
scopus_data/ATB_author_abstract_funding.csv 1102
scopus_data/ATX_author_abstract_funding.csv 68
scopus_data/ECA_author_abstract_funding.csv 3453
scopus_data/ECX_author_abstract_funding.csv 204
scopus_data/JEM_author_abstract_funding.csv 1439
scopus_data/JFE_author_abstract_funding.csv 6937
scopus_data/JHR_author_abstract_funding.csv 2012
scopus_data/JLE_author_abstract_funding.csv 1457
scopus_data/JLO_author_abstract_funding.csv 1328
scopus_data/JOF_author_abstract_funding.csv 5446
scopus_data/JOL_author_abstract_funding.csv 1761
scopus_data/JPE_author_abstract_funding.csv 2799
scopus_data/QJE_author_abstract_funding.csv 2923
scopus_data/RES_author_abstract_funding.csv 2868
scopus_data/RFS_author_abstract_funding.csv 4902
scopus_data/RJE_author_abstract_funding.csv 2212
Author-Abstract observations: 52461
Unique Authors (from author-abstract data): 22751


In [16]:
print('READING IN EDITOR DATA')
editor_excel_path = 'editors_data/editors_database.xlsx'
df = pd.read_excel(editor_excel_path)
df['Year_Count'] = df['Years'].str.len()
df1 =df[df['Year_Count']!=9] 
df["Years"] = df["Years"].str.strip()
df = df[df['Year_Count']>=9] 
df["start"] = pd.to_numeric(df["Years"].str.slice(stop=4))
df["stop"] = pd.to_numeric(df["Years"].str.slice(start=5))
df["tenure"] = df["stop"]-df["start"]+1
df = df.loc[df.index.repeat(df.tenure)]
df['dup_number'] = df.groupby(['Editor Name','Years']).cumcount()+1
df["Year"] = df["start"]+df["dup_number"]-1
df1["Year"] = pd.to_numeric(df1["Years"])
df = df.append(df1)
df = df[df["Year"]>=1990]
df = df[['Journal','Editor Name','Year','Editor Position']]
df


READING IN EDITOR DATA


C:\Users\Joshualevy\AppData\Local\Temp\ipykernel_31652\3595274863.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df1)


,Journal,Editor Name,Year,Editor Position
6,AER,Orley Ashenfelter,1990,Editor
6,AER,Orley Ashenfelter,1991,Editor
6,AER,Orley Ashenfelter,1992,Editor
6,AER,Orley Ashenfelter,1993,Editor
6,AER,Orley Ashenfelter,1994,Editor
...,...,...,...,...
720,JEMS,Daniel F. Spulber,1993,Editor
721,JEMS,Daniel F. Spulber,1992,Editor
729,JLE,Randall S. Kroszner,2001,Editor
738,JLE,Adam Chilton,2022,Editor


In [6]:
print('CLEANING EDITOR DATA')
df_editor = pd.DataFrame(df['Editor Name'].unique())
df_editor.columns = ['Full Name']
df_editor['parser_object'] = df_editor['Full Name'].map(lambda x: HumanName(x))
df_editor['First'] = df_editor['parser_object'].map(lambda x: x.first)
df_editor['Last'] = df_editor['parser_object'].map(lambda x: x.last)
df_editor['middle_initial'] = df_editor['parser_object'].map(lambda x: x.middle)
df_editor['first_initial'] = df_editor['First'].map(lambda x: x[0] + '.')

df_editor

CLEANING EDITOR DATA


,Full Name,parser_object,First,Last,middle_initial,first_initial
0,Orley Ashenfelter,Orley Ashenfelter,Orley,Ashenfelter,,O.
1,Ben S. Bernanke,Ben S. Bernanke,Ben,Bernanke,S.,B.
2,Robert A. Moffitt,Robert A. Moffitt,Robert,Moffitt,A.,R.
3,Pinelopi K. Goldberg,Pinelopi K. Goldberg,Pinelopi,Goldberg,K.,P.
4,Robert H. Haveman,Robert H. Haveman,Robert,Haveman,H.,R.
...,...,...,...,...,...,...
519,Thomas G. McGuire,Thomas G. McGuire,Thomas,McGuire,G.,T.
520,Rajiv Lal,Rajiv Lal,Rajiv,Lal,,R.
521,Michael H. Riordan,Michael H. Riordan,Michael,Riordan,H.,M.
522,Adam Chilton,Adam Chilton,Adam,Chilton,,A.


In [7]:
print('MATCHING')
authors_editors_merge = pd.merge(authors_abstracts_df, df_editor, how='outer',
left_on = ['sc_author_given_name', 'sc_author_last_name'], right_on = ['First', 'Last'],
indicator=True)
len(authors_editors_merge[authors_editors_merge._merge == 'both'])
print('Number of matched editor-observations: {}'.format(len(authors_editors_merge[authors_editors_merge._merge == 'both'])))

MATCHING
Number of matched editor-observations: 3231


In [8]:
editors_authors_matched = authors_editors_merge[authors_editors_merge._merge == 'both']
editors_authors_matched = editors_authors_matched.drop_duplicates(subset=['sc_author_id', 'First', 'Last'])
editors_authors_matched = editors_authors_matched[['Full Name', 'sc_author_id', 'sc_author_given_name', 'sc_author_last_name', 'sc_author_indexed_name', 'First', 'Last']]
editors_authors_matched

,Full Name,sc_author_id,sc_author_given_name,sc_author_last_name,sc_author_indexed_name,First,Last
44,Chad Syverson,6.602936e+09,Chad,Syverson,Syverson C.,Chad,Syverson
76,Rema Hanna,3.518514e+10,Rema,Hanna,Hanna R.,Rema,Hanna
118,Susan Athey,6.603691e+09,Susan,Athey,Athey S.,Susan,Athey
170,Frank Verboven,6.603826e+09,Frank,Verboven,Verboven F.,Frank,Verboven
220,Magne Mogstad,1.576562e+10,Magne,Mogstad,Mogstad M.,Magne,Mogstad
...,...,...,...,...,...,...,...
51554,Scott E. Carrell,1.605217e+10,Scott,Carrell,Carrell S.,Scott,Carrell
51677,Jeffrey L. Coles,7.102342e+09,Jeffrey,Coles,Coles J.,Jeffrey,Coles
52226,Daniel F. Spulber,6.603758e+09,Daniel,Spulber,Spulber D.,Daniel,Spulber
52279,Robert Porter,7.401897e+09,Robert,Porter,Porter R.,Robert,Porter


In [9]:
editors_authors_unmatched = authors_editors_merge[authors_editors_merge._merge == 'right_only']
editors_authors_unmatched = editors_authors_unmatched.drop_duplicates(subset=['sc_author_id', 'First', 
'Last'])
editors_authors_unmatched = editors_authors_unmatched[['Full Name', 'sc_author_id', 'sc_author_given_name', 'sc_author_last_name', 'sc_author_indexed_name', 'First', 'Last']]
editors_authors_unmatched

,Full Name,sc_author_id,sc_author_given_name,sc_author_last_name,sc_author_indexed_name,First,Last
52572,Bennett T. McCallum,NaN,NaN,NaN,NaN,Bennett,McCallum
52573,John Y. Campbell,NaN,NaN,NaN,NaN,John,Campbell
52575,Kenneth D. West,NaN,NaN,NaN,NaN,Kenneth,West
52577,R. Preston McAfee,NaN,NaN,NaN,NaN,R.,McAfee
52578,Matthew D. Shapiro,NaN,NaN,NaN,NaN,Matthew,Shapiro
...,...,...,...,...,...,...,...
52707,James E. Rauch,NaN,NaN,NaN,NaN,James,Rauch
52708,Pablo T. Spiller,NaN,NaN,NaN,NaN,Pablo,Spiller
52709,Ching-to Albert Ma,NaN,NaN,NaN,NaN,Ching-to,Albert
52710,Thomas G. McGuire,NaN,NaN,NaN,NaN,Thomas,McGuire


In [10]:
def queryConstruction(firstname, lastname):
    author_search_params = {
        'httpAccept' : 'application/json',
        'query' : 'AUTHFIRST({}) AND AUTHLASTNAME({}) AND SUBJAREA(ECON)'.format(firstname, lastname),
        'alias' : 'true'
    }
    return author_search_params

In [11]:
manual_web_search_count = 0
deep_name_investigation = 0 
for index, observation in editors_authors_unmatched.iterrows():
    firstname = observation.First
    lastname = observation.Last

    response = requests.get(
        url = 'https://api.elsevier.com/content/search/author',
        headers=req_headers,
        params=queryConstruction(firstname, lastname)
    )
    
    if response.status_code == 200:
        total_results_count = response.json()['search-results']['opensearch:totalResults']

        if total_results_count == "1": 
            result_object = response.json()['search-results']['entry'][0]
            scopus_author_id_string = result_object['dc:identifier']
            scopus_author_id = int(scopus_author_id_string[10:])
            editors_authors_unmatched.loc[index, 'sc_author_id'] = scopus_author_id

        elif total_results_count == '0':
            print('{} {} deserves much more attention. Do a name check.'.format(firstname, lastname))
            deep_name_investigation +=1

        else:
            print('Find SCOPUS ID for {} {} by manual web search.'.format(firstname, lastname))
            manual_web_search_count += 1

    time.sleep(0.6)
            
print('{} authors require some SCOPUS ID manual web search (disambiguate between multiple IDs)'.format(manual_web_search_count))
print('{} authors require deep investigation. Check names as nameparser is not perfect.'.format(deep_name_investigation))

Find SCOPUS ID for John Campbell by manual web search.
Find SCOPUS ID for R. McAfee by manual web search.
Find SCOPUS ID for Matthew Shapiro by manual web search.
Find SCOPUS ID for B. Bernheim by manual web search.
Jeff Ely deserves much more attention. Do a name check.
Find SCOPUS ID for Charles Jones by manual web search.
Find SCOPUS ID for Robert Anderson by manual web search.
Find SCOPUS ID for Donald Andrews by manual web search.
Find SCOPUS ID for Robert Becker by manual web search.
Find SCOPUS ID for V.V. Chari by manual web search.
Find SCOPUS ID for Manfred Deistler by manual web search.
Find SCOPUS ID for James Hamilton by manual web search.
Find SCOPUS ID for Joel Horowitz by manual web search.
Yuzo Hosova deserves much more attention. Do a name check.
Find SCOPUS ID for Soren Johansen by manual web search.
David Levien deserves much more attention. Do a name check.
Bart Lipman deserves much more attention. Do a name check.
Find SCOPUS ID for Adrian Pagan by manual web sear

In [12]:
matched_and_unmatched_df = pd.concat([editors_authors_matched, editors_authors_unmatched], ignore_index=True)
matched_and_unmatched_df


,Full Name,sc_author_id,sc_author_given_name,sc_author_last_name,sc_author_indexed_name,First,Last
0,Chad Syverson,6.602936e+09,Chad,Syverson,Syverson C.,Chad,Syverson
1,Rema Hanna,3.518514e+10,Rema,Hanna,Hanna R.,Rema,Hanna
2,Susan Athey,6.603691e+09,Susan,Athey,Athey S.,Susan,Athey
3,Frank Verboven,6.603826e+09,Frank,Verboven,Verboven F.,Frank,Verboven
4,Magne Mogstad,1.576562e+10,Magne,Mogstad,Mogstad M.,Magne,Mogstad
...,...,...,...,...,...,...,...
497,James E. Rauch,NaN,NaN,NaN,NaN,James,Rauch
498,Pablo T. Spiller,NaN,NaN,NaN,NaN,Pablo,Spiller
499,Ching-to Albert Ma,7.402925e+09,NaN,NaN,NaN,Ching-to,Albert
500,Thomas G. McGuire,NaN,NaN,NaN,NaN,Thomas,McGuire


In [17]:
editors_scopus_id = pd.merge(df, matched_and_unmatched_df, how='outer', left_on='Editor Name', right_on='Full Name', indicator=True)
editors_scopus_id = editors_scopus_id.drop(columns=['sc_author_given_name', 'sc_author_last_name', 'First', 'Last', '_merge'])
editors_scopus_id.to_csv('editors_data/editors_database_pre_hand_addition.csv', index=False, encoding='utf-8')

In [14]:
def constructAliases(firstname, lastname, initials):
    #First Last
    x = firstname + ' ' + lastname
    # Initials Last
    y = initials + ' ' + lastname
    return [x, y]


In [15]:
# aliases_dict = {}
# for i, author_object in enumerate(response.json().get('search-results').get('entry')):
#     scopus_id = author_object.get('dc:identifier').split(':')[1]
#     print(scopus_id)
#     aliases_dict[scopus_id] = []
#     author_object_preferred_name_obj = author_object.get('preferred-name')
#     print('Preferred name fields: {}'.format(i))
#     surname = author_object_preferred_name_obj.get('surname')
#     givenname = author_object_preferred_name_obj.get('given-name')
#     initials = author_object_preferred_name_obj.get('initials')
#     print('\t{}'.format(surname))
#     print('\t{}'.format(givenname))
#     print('\t{}'.format(initials))
#     aliases_dict[scopus_id].append(constructAliases(givenname, surname, initials))

#     author_object_name_variant_obj = author_object.get('name-variant')
#     if author_object_name_variant_obj == None:
#         print('nothing to see here')
#     elif type(author_object_name_variant_obj) == list: 
#         for j, name_variant_object in enumerate(author_object_name_variant_obj):


#             variant_surname = name_variant_object.get('surname')
#             variant_givenname = name_variant_object.get('given-name')
#             variant_initials = name_variant_object.get('initials')
#             print('Name variant fields: {}-{}'.format(i,j))
#             print('\t{}'.format(variant_surname))
#             print('\t{}'.format(variant_givenname))
#             print('\t{}'.format(variant_initials))
#             aliases_dict[scopus_id].append(constructAliases(variant_givenname, variant_surname, variant_initials))
    
#     aliases_dict[scopus_id] = set(sum(aliases_dict[scopus_id], []))
#     print(aliases_dict)

In [33]:
def generateHandMatchCSV():
    auto_matched_scopus_ids_df = pd.read_csv('editors_data/editors_database_pre_hand_addition.csv')

    to_be_hand_matched_df = auto_matched_scopus_ids_df.drop_duplicates(subset=['Editor Name'])
    to_be_hand_matched_df = to_be_hand_matched_df[to_be_hand_matched_df.sc_author_id.isna()]
    to_be_hand_matched_df = to_be_hand_matched_df.drop(columns=['Year', 'Full Name', 'sc_author_indexed_name'])

    to_be_hand_matched_df.to_csv('editors_data/editors_database_hand_addition_templated.csv', index=False, encoding='utf-8')
    return to_be_hand_matched_df



In [34]:
generateHandMatchCSV()

,Journal,Editor Name,Editor Position,sc_author_id
49,AER,John Y. Campbell,Co-Editor,NaN
60,AER,R. Preston McAfee,Co-Editor,NaN
76,AER,Matthew D. Shapiro,Co-Editor,NaN
101,AER,B. Douglas Bernheim,Co-Editor,NaN
229,AER,Jeff Ely,Co-Editor,NaN
...,...,...,...,...
3268,JEMS,Simon Parker,Co-Editor,NaN
3353,JEMS,Juan D. Carrillo,Co-Editor,NaN
3359,JEMS,James E. Rauch,Editor,NaN
3360,JEMS,Pablo T. Spiller,Editor,NaN


In [ ]:
def includeHandMatches():
    auto_matched_df = pd.read_csv('editors_data/editors_database_pre_hand_addition.csv')
    hand_matches_df = pd.read_csv('editors_data/editors_database_hand_addition_templated.csv')
    # # auto_matched_df_auto_matches_only = auto_matched_df[auto_matched_df['sc_author_id'].notna()]
    # auto_matched_df_auto_missing_only = auto_matched_df[auto_matched_df['sc_author_id'].isna()]

    # auto_matched_df_auto_missing_only = auto_matched_df_auto_missing_only.drop_duplicates
    # Processing of hand-matched SCOPUS IDs. 
    
    auto_and_hand_matched_df = pd.merge(auto_matched_df, hand_matches_df, how='outer', on='Editor Name', indicator=True)


    auto_and_hand_matched_df.to_csv('editors_data/editors_database_post_hand_addition.csv', index=False, encoding='utf-8')

    return auto_matched_df
